In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from pickle import dump

In [2]:
#fig, ax = plt.subplots(1, figsize=(13, 7))
#ax.plot(train_data[objective], label='Train', linewidth=2)
#ax.plot(test_data[objective], label='Test', linewidth=2)
#ax.set_ylabel('Price USD', fontsize=14)
#ax.set_title('', fontsize=16)
#ax.legend(loc='best', fontsize=16)

In [3]:
def trainingsteps(objective, partial_name, train, train_data, test_data):
    #Seleccone el onjetivo
    datacol = train[objective].values
    traincol = train_data[objective].values
    testcol = test_data[objective].values
    #Reshape
    shaped_data = np.reshape(datacol, (-1,1))
    train_shaped = np.reshape(traincol, (-1,1))
    test_shaped = np.reshape(testcol, (-1,1))
    #Reescalar el objetivo
    sc = MinMaxScaler(feature_range=(0,1))
    sc.fit(shaped_data)
    dump(sc, open('models/'+partial_name+'_scaler.pkl', 'wb'))
    train_scaled_data = sc.transform(train_shaped)
    test_scaled_data = sc.transform(test_shaped)
    return train_scaled_data, test_scaled_data

def seriesdefinition(time_steps,lendata,data):
    X = []
    y = []

    for i in range(time_steps, lendata):
        X.append(data[i-time_steps:i,0])
        y.append(data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [4]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

def modeldefinition():
    model = Sequential()
    model.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(optimizer='adam', loss='mse')
    return model

In [5]:
def training(X_train,y_train,partialname):
    model = modeldefinition()
    callback = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    checkpoint_callback = ModelCheckpoint('models/'+partialname+'.ckpt', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=1, callbacks=[callback,checkpoint_callback], shuffle=True)

In [6]:
coins = ["Dogecoin","Litecoin","Bitcoin","XRP","Stellar","Monero","NEM","Ethereum","Cardano"]
model = "LSTM"
#coin = "Bitcoin"
for coin in coins:
    partial_name = model+ " " + coin
    objective = "Close"
    train = pd.read_csv('archive/coin_'+coin+'.csv')
    ##20 entrenamiento-80 pruebas
    split_row = len(train) - int(0.2 * len(train))
    train_data = train.iloc[:split_row]
    test_data = train.iloc[split_row:]
    train_scaled_data, test_scaled_data = trainingsteps(objective, partial_name, train, train_data, test_data)
    X_train, X_test, y_train, y_test = seriesdefinition(50,len(train_data),train_scaled_data)
    training(X_train,y_train,partial_name)

Epoch 1/100
52/54 [===========================>..] - ETA: 0s - loss: 1.4826e-05
Epoch 1: val_loss improved from inf to 0.00001, saving model to models\LSTM Dogecoin.ckpt


INFO:tensorflow:Assets written to: models\LSTM Dogecoin.ckpt\assets


INFO:tensorflow:Assets written to: models\LSTM Dogecoin.ckpt\assets


54/54 [==============================] - 9s 138ms/step - loss: 1.4396e-05 - val_loss: 5.5042e-06
Epoch 2/100
52/54 [===========================>..] - ETA: 0s - loss: 2.4355e-06
Epoch 2: val_loss improved from 0.00001 to 0.00000, saving model to models\LSTM Dogecoin.ckpt


INFO:tensorflow:Assets written to: models\LSTM Dogecoin.ckpt\assets


...

INFO:tensorflow:Assets written to: models\LSTM Cardano.ckpt\assets


27/27 [==============================] - 7s 263ms/step - loss: 1.5451e-04 - val_loss: 5.3145e-05
Epoch 47/100
25/27 [==========================>...] - ETA: 0s - loss: 1.5078e-04
Epoch 47: val_loss did not improve from 0.00005
27/27 [==============================] - 1s 23ms/step - loss: 1.4550e-04 - val_loss: 7.0496e-05
Epoch 48/100
25/27 [==========================>...] - ETA: 0s - loss: 1.5536e-04
Epoch 48: val_loss improved from 0.00005 to 0.00005, saving model to models\LSTM Cardano.ckpt


INFO:tensorflow:Assets written to: models\LSTM Cardano.ckpt\assets


INFO:tensorflow:Assets written to: models\LSTM Cardano.ckpt\assets


27/27 [==============================] - 7s 265ms/step - loss: 1.5119e-04 - val_loss: 5.2379e-05
